# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid

In [2]:
# fictional grid parameters

n_node_per_feeder = 3
n_feeder_small = 100
n_feeder_big = 2

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95


# Big Network

In [3]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder_big,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

pgm_model = pgm.PowerGridModel(pgm_dataset)